# ABC анализ в Аптечной сети 

## провести многомерный ABC-анализ по 3 признакам:

- количество проданных штук товара
- сумма продаж
- месячная оборачиваемость

Загружаем и решаем задачу с помощью postgres SQL

1) Агрегируем данные
   - по количеству на товар
   - по выручке с учетом скидки на товар
   - по среднедневное количество продаж умноженное на 30 дней

In [ ]:
with
group_table as (
  select
    ds.dr_ndrugs
    , sum(ds.dr_kol) as amount
    , sum(ds.dr_croz - ds.dr_sdisc) as profit_with_discount 
    , SUM(ds.dr_kol) / COUNT(DISTINCT dr_dat) * 30 AS month_average  
  from 
    drugs_simulative ds 
  where 
    ds.dr_ndrugs != ''
  group by 
    dr_ndrugs 
)

2. Расчитываем накопительную сумму по перечисленным выше полям 

In [ ]:
, cumsum_table as (
  select 
    dr_ndrugs 
    , round(sum(amount) over(order by amount DESC) /  sum(amount) over() * 100.0, 2) as cumsum_count
    , round(sum(profit_with_discount ) over(order by profit_with_discount DESC) / sum(profit_with_discount ) over() * 100.0, 2) as cumsum_profit_with_disc
    , round(sum(month_average) over(order by month_average DESC) / sum(month_average ) over() * 100.0, 2) as cumsum_month_average

3. Производим ABC анализ с учетом данных накопительной суммы по стандартному закону Парето 80/20

In [ ]:
, case
      when sum(amount) over(order by amount DESC) /  sum(amount) over() * 100.0 <= 80 then 'A'
      when sum(amount) over(order by amount DESC) /  sum(amount) over() * 100.0 <= 95 then 'B'
      else 'C'
    end as ABC_count
    , case
      when sum(profit_with_discount) over(order by profit_with_discount DESC) /  sum(profit_with_discount) over() * 100.0 <= 80 then 'A'
      when sum(profit_with_discount) over(order by profit_with_discount DESC) /  sum(profit_with_discount) over() * 100.0 <= 95 then 'B'
      else 'C'
    end as ABC_profit_with_disc
    , case
      when sum(month_average) over(order by month_average DESC) /  sum(month_average) over() * 100.0 <= 80 then 'A'
      when sum(month_average) over(order by month_average DESC) /  sum(month_average) over() * 100.0 <= 95 then 'B'
      else 'C'
    end as ABC_month_average
  from 
    group_table
)

4. Выводим произведенный ABC анализ в порядке: количество проданных штук товара, сумма продаж, месячная оборачиваемость.
- отсортированный по: сумма продаж, количество проданных штук товара, месячная оборачиваемость.

In [ ]:
select
  dr_ndrugs 
  , ABC_count || ABC_profit_with_disc || ABC_month_average as count_profit_monthturnover
from
  cumsum_table 
order by
  ABC_profit_with_disc
  , ABC_count
  , abc_month_average ;

##  решение целиком 

In [ ]:
with
group_table as (
  select
    ds.dr_ndrugs
    , sum(ds.dr_kol) as amount
    , sum(ds.dr_croz - ds.dr_sdisc) as profit_with_discount 
    , SUM(ds.dr_kol) / COUNT(DISTINCT dr_dat) * 30 AS month_average  
  from 
    drugs_simulative ds 
  where 
    ds.dr_ndrugs != ''
  group by 
    dr_ndrugs 
)
, cumsum_table as (
  select 
    dr_ndrugs 
    , round(sum(amount) over(order by amount DESC) /  sum(amount) over() * 100.0, 2) as cumsum_count
    , round(sum(profit_with_discount ) over(order by profit_with_discount DESC) / sum(profit_with_discount ) over() * 100.0, 2) as cumsum_profit_with_disc
    , round(sum(month_average) over(order by month_average DESC) / sum(month_average ) over() * 100.0, 2) as cumsum_month_average
    , case
      when sum(amount) over(order by amount DESC) /  sum(amount) over() * 100.0 <= 80 then 'A'
      when sum(amount) over(order by amount DESC) /  sum(amount) over() * 100.0 <= 95 then 'B'
      else 'C'
    end as ABC_count
    , case
      when sum(profit_with_discount) over(order by profit_with_discount DESC) /  sum(profit_with_discount) over() * 100.0 <= 80 then 'A'
      when sum(profit_with_discount) over(order by profit_with_discount DESC) /  sum(profit_with_discount) over() * 100.0 <= 95 then 'B'
      else 'C'
    end as ABC_profit_with_disc
    , case
      when sum(month_average) over(order by month_average DESC) /  sum(month_average) over() * 100.0 <= 80 then 'A'
      when sum(month_average) over(order by month_average DESC) /  sum(month_average) over() * 100.0 <= 95 then 'B'
      else 'C'
    end as ABC_month_average
  from 
    group_table
)
select
  dr_ndrugs 
  , ABC_count || ABC_profit_with_disc || ABC_month_average as count_profit_monthturnover
from
  cumsum_table 
order by
  ABC_profit_with_disc
  , ABC_count
  , abc_month_average ;